## 벡터화

### Bag of words / DTM(Document-Term Matrix)
- 단어의 순서는 고려하지 않고 빈도만을 고려하여 벡터화
- bag of words를 이용해서 문서 간의 유사도를 비교하는 행렬을 만든 것을 DTM이라고 함
    - DTM은 문서를 행으로, 단어를 열로 구성한 행렬
    - 대부분의 값이 0인 희소 벡터
- 별도의 전처리가 필요(?) - 중복, 불필요한 용어

### TF-IDF
- DTM은 두 문서에서 공통적으로 등장하는 단어가 많을 수록 유사하다고 판단 될 수 있으나, 이는 별로 중요하지 않은 단어에서 겹칠경우 유사도가 높게 판단되는 단점이 있을 수 있음
- TF-IDF는 이를 보완하여 단어마다 중요 가중치를 다르게 주는 방법 적용
- 그러나 여전히 희소벡터에 해당

### 원-핫 인코딩(one-hot encoding)
- 원 핫 인코딩을 하기 위해서는 단어장부터 만들어야 함
- 이 후 1부터 V까지 고유 정수를 부여 -> 이 숫자는 인덱스 역할을 함
- 관례적으로 빈도수가 높은 단어들부터 낮은 숫자를 부여

## 벡터화 실습 : 원-핫 인코딩 구현

In [ ]:
$ pip install konlpy

In [ ]:
import re
from konlpy.tag import Okt
from collections import Counter

In [ ]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

In [ ]:
# 전처리 
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

In [ ]:
# 토큰화 
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

In [ ]:
# 단어장 만들기
# 빈도수가 높은 단어일수록 낮은 정수를 부여
vocab = Counter(tokens) # 파이썬 collections.Counter
print(vocab)

In [ ]:
vocab['임금님']

In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

In [ ]:
# 상위 5개의 단어에서 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

In [ ]:
# 원-핫 벡터 만들기
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index))
    index = word2index[word]
    one_hot_vector[index-1] = 1
    return one_hot_vector

In [ ]:
one_hot_encoding("임금님", word2idx)

In [ ]:
# 케라스를 통한 원-핫 인코딩(one-hot encoding)
from tensorflow.keras.preprocessing.text import Tokenizer # 단어장 만들기
from tensorflow.keras.utils import to_categorical # 원-핫 인코딩

In [ ]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

In [ ]:
t = Tokenizer()
t.fit_on_texts(text) # 단어장 만들기 
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

In [ ]:
vocab_size = len(t.word_index) + 1 # 단어장 크기를 vocab_size 변수에 저장
# vocab_size를 구할 때 1을 더해주는 이유는, 
# 실제로 자연어 처리를 할 때는 0번 단어가 특별 토큰으로 단어장에 추가되는 경우가 많기 때문
# 주로 0번은 padding 토큰

In [ ]:
# 단어장에 속한 단어들로 구성된 텍스트 시퀀스는 케라스 토크나이저를 통해 정수 시퀀스로 변환
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

In [ ]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

### 워드 임베딩
- 희소벡터는 차원의 저주라는 문제가 있음
- 벡터 간 유사도를 구하는 방법으로는 대표적으로 내적(inner product)
- 한 단어를 길이가 비교적 짧은 밀집 벡터로 나타낸다.
- 그런데 이 밀집 벡터는 단어가 갖는 의미나 단어 간의 관계 등을 어떤 식으로든 내포하고 있다.
- 이렇게 만들어진 밀집 벡터를 임베딩 벡터라고 말한다. 

- 임베딩 벡터의 값은 어떤 방식을 통해서 얻어지나요? 훈련 데이터로부터 어떤 모델을 학습하는 과정에서 '자동'으로 얻어지는데, 주로 언어 모델(Language Model)을 학습하는 가운데 얻어짐.

- A Neural Probabilistic Language Model
- 워드 임베딩은 2003년 요슈아 벤지오(Yoshua Bengio) 교수가 NPLM(Neural Probabilistic Language Model) 이란 모델을 통해 제안됨.
- 2013년 구글은 NPLM을 개선하여 정밀도와 속도를 향상시킨 Word2Vec을 제안됨. 
- Word2Vec 이후로 FastText 나 GloVe 등과 같은 임베딩 방법이 추가로 제안됨

#### Word2Vec (1) 분포 가설
- Word2Vec은 앞서 말했듯이 단어를 벡터로 표현하는 방법의 일종으로 저차원으로 이루어져 있고, 단어의 의미를 여러 차원에 분산하여 표현한 벡터.
- Word2Vec의 핵심 아이디어는 분포 가설(distributional hypothesis) 을 따름.
- 분포 가설에 따르는 Word2Vec은 같이 등장하는 경향이 적은 단어들에 비해 '강아지', '애교, '귀여운'과 같은 단어들을 상대적으로 유사도가 높은 벡터로 만든다.

#### Word2Vec (2) CBoW
- Word2Vec에는 크게 CBoW와 Skip-gram라는 두 가지 방법이 있다. 
- CBoW는 주변에 있는 단어들을 통해 중간에 있는 단어들을 예측하는 방법. 
- Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법.


예문 : "I like natural language processing."
- CBoW는 중간에 있는 단어를 예측하는 방법이므로 {"i", "like", "language", "processing"}으로부터 "natural"을 예측하는 것은 CBoW가 하는 일.
- 이때 예측해야 하는 단어 "natural"을 중심 단어(center word) 라고 하고, 예측에 사용되는 단어들을 주변 단어(context word) 라고 함.
- 중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지를 결정를 결정했다면, 그 범위를 윈도우(window) 라고 함.

[슬라이딩 윈도우]
https://medium.com/analytics-vidhya/word2vector-using-gensim-e055d35f1cb4
- 윈도우 크기를 정했다면, 윈도우를 계속 움직여서 주변 단어와 중심 단어를 바꿔가며 학습을 위한 데이터 셋을 만들 수 있는데, 이 방법을 슬라이딩 윈도우(sliding window) 라고 함.

- CBoW는 출력층의 벡터를 중심 단어의 원-핫 벡터와의 손실(loss)을 최소화 하도록 학습함.
- CBoW는 Out-of-vocabulary(단어 집합에 없는 단어), Polysemy(다의어), 문맥 파악의 한계가 있음.


### 네거티브 샘플링(negative sampling)
- 대체적으로 Word2Vec를 사용할 때는 SGNS(Skip-Gram with Negative Sampling) 을 사용함.
- Skip-gram을 사용하면서 네거티브 샘플링(Negative Sampling) 이란 방법도 사용한다는 의미.
- Word2Vec의 구조는 연산량이 지나치게 많아 실제로 사용하기 어려움.
- 거티브 샘플링은 연산량을 줄이기 위해서 소프트맥스 함수를 사용한 V개 중 1개를 고르는 다중 클래스 분류 문제 를 시그모이드 함수를 사용한 이진 분류 문제 로 바꿈.
- 중심 단어와 주변 단어를 입력값으로 받아 이 두 단어가 정말로 이웃 관계면(실제로 중심 단어와 주변 단어의 관계면) 1을 또는 0을 출력하는 문제로 바꿈.
- 즉, 기존의 다중 분류 문제에서 이진 분류 문제로 바뀐 것.


### CBoW와 Skip-gram 차이
CBoW는 주변 단어(Context Words)를 가지고 중심 단어(Target Word)를 예측하는 모델.
즉, 주어진 문맥에서 중심 단어를 예측하는 것이 목적.

Skip-gram은 중심 단어를 가지고 주변 단어를 예측하는 모델.
즉, 중심 단어를 입력으로 받아서 주어진 문맥에서 주변 단어를 예측하는 것이 목적.

## Word2Vec (4) 영어 Word2Vec 실습과 OOV 문제
- Word2Vec을 별도로 구현할 필요없이 파이썬의 gensim 패키지를 통해 이미 구현된 Word2Vec 모델을 사용할 수 있음.
- 사용할 훈련 데이터는 NLTK에서 제공하는 코퍼스이며, gensim 패키지는 토픽 모델링을 위한 NLP 패키지임.  

$ pip install nltk  

$ pip install gensim   

In [ ]:
import nltk
nltk.download('abc')
nltk.download('punkt')

In [ ]:
from nltk.corpus import abc
corpus = abc.sents()

In [ ]:
print(corpus[:3])

In [ ]:
print('코퍼스의 크기 :',len(corpus))

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

위 코드에서 각 파라미터의 의미

- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않음.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

Word2Vec는 입력한 단어에 대해서 가장 코사인 유사도가 높은 단어들을 출력하는 model.wv.most_similar를 지원.

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

In [ ]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")
print("모델  load 완료!")

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

In [ ]:
## Word2Vec의 OOV 문제
loaded_model.most_similar('overacting') # 에러남

## FastText
- 페이스북에서 개발한 FastText는 Word2Vec 이후에 등장한 워드 임베딩 방법
- 메커니즘 자체는 Word2Vec을 그대로 따르고 있지만, 문자 단위 n-gram(character-level n-gram) 표현을 학습한다
- FastText는 단어 내부의 내부 단어(subwords)들을 학습한다는 아이디어를 가지고 있습니다.

n = 3인 경우   

<pa, par, art, rti, tia, ial, al>, \<partial>   

- 실제 사용할 때는 n의 최솟값과 최댓값으로 범위를 설정할 수 있는데, gensim 패키지에서는 기본값으로 각각 3과 6으로 설정되어 있음.
- FastText는 Word2Vec과 달리 OOV와 오타에 강건하다(robust)

In [ ]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

fasttext_model.wv.most_similar('overacting')

In [ ]:
fasttext_model.wv.most_similar('memoryy')

## GloVe
- 글로브(Global Vectors for Word Representation, GloVe) 는 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론.
- 워드 임베딩의 두 가지 접근 방법인 카운트 기반과 예측 기반 두 가지 방법을 모두 사용함.
- 밀집 표현(dense representation)으로 임베딩 하는 방법이 LSA(Latent Semantic Analysis)
- 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)


In [ ]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

In [ ]:
glove_model.most_similar('overacting')

In [ ]:
glove_model.most_similar('memoryy')